**Importing kaggle library to download and access Data Sets**

In [ ]:
!pip install kaggle # to install kaggle library to import datasets

**Importing the Dependencies**

In [ ]:
import os  #to interact with operating system,creating folders,setting fille paths
import json  #to read and write data in json format, kaggle api key that store credentials is also stored in json format

from zipfile import ZipFile  # importing utility that helps to unzip the compressed downloaded data sets
import pandas as pd  # for dataset loading and data analysis
from sklearn.model_selection import train_test_split  # to split data into train and test
from tensorflow.keras.models import Sequential  # to create sequential models that are linearly stacked as input and hidden and output layers
from tensorflow.keras.layers import Dense, Embedding, LSTM
# dense refers to fully connected neural network
# embedding help to convert words or tokens to numerical vector representation
# lstm model
from tensorflow.keras.preprocessing.text import Tokenizer  #to tokenize/split data into tokens base on words or sentences
from tensorflow.keras.preprocessing.sequence import pad_sequences  #ensures all input sequences have same shape

**Data Collection- Kaggle API**

In [ ]:
kaggle_dictionary = json.load(open("kaggle.json")) # loading json format file,opening and saving in variable

In [ ]:
kaggle_dictionary.keys()  # to check keys

dict_keys(['username', 'key'])

In [ ]:
# setup kaggle credentials as environment variables
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"] # os.environ is a dictiory object in python to access environment variables
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"] #

In [ ]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
  0% 0.00/25.7M [00:00<?, ?B/s]
100% 25.7M/25.7M [00:00<00:00, 762MB/s]


In [ ]:
!ls  #to check list

imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [ ]:
# Unzip the dataset file

In [ ]:
with ZipFile('/content/imdb-dataset-of-50k-movie-reviews.zip','r') as zip_ref:
  # with ZipFile utility we will provide path to unzip the zipped file, r commands to read a file and we will use alias as zip_ref
  zip_ref.extractall()  #extracting all the compressed data from zip file


In [ ]:
!ls # cheking list of files on colab

'IMDB Dataset.csv'			 kaggle.json
 imdb-dataset-of-50k-movie-reviews.zip	 sample_data


**Loading Dataset**

In [ ]:
df=pd.read_csv('/content/IMDB Dataset.csv')

In [ ]:
df.shape

(50000, 2)

In [ ]:
df.head()

review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive

In [ ]:
df.tail()

review sentiment
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

In [ ]:
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [ ]:
df.replace({'sentiment':{'positive':1,'negative':0}},inplace=True)

/tmp/ipython-input-18-2906463383.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'sentiment':{'positive':1,'negative':0}},inplace=True)


In [ ]:
df.head()

review  sentiment
0  One of the other reviewers has mentioned that ...          1
1  A wonderful little production. <br /><br />The...          1
2  I thought this was a wonderful way to spend ti...          1
3  Basically there's a family where a little boy ...          0
4  Petter Mattei's "Love in the Time of Money" is...          1

**Spliting the data into  train and test**

In [ ]:
# For now I am going to split the whole dataframe into train data and tet data
train_data,test_data= train_test_split(df, test_size=0.2, random_state=42)
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


**Data Preprocessing**

In [ ]:
# Tokenizing the text data
tokenizer= Tokenizer(num_words=5000)
#since we dont know vocablury size,so we take whole training data,num_words=5000 means tokenizer will select 5k most frequent words as vocablury and ignore all other words
# modern tokenizer not only just split text but also convert them to a unique integer representing the word
tokenizer.fit_on_texts(train_data['review']) #training tokenizer on direct texts data of review column
X_train= pad_sequences(tokenizer.texts_to_sequences(train_data['review']),maxlen=200)
X_test=pad_sequences(tokenizer.texts_to_sequences(test_data['review']),maxlen=200)
# The flow:
# 1) tokenizer.fit will train on text and get you the vocablury of most common 5000 words
# 2) then tokenizer.text_to_sequence will convert them into unique integers
# 3) then pad_sequence will ensure that each review/data point in review column has same length,
# 4) max_ len= 200 means every review/datapoint has 200 tokens equally
print(X_train)
print(X_test)


[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]
[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [ ]:
Y_train= train_data['sentiment']
Y_test= test_data['sentiment']

In [ ]:
print(Y_train)
print(Y_test)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64
33553    1
9427     1
199      0
12447    1
39489    0
        ..
28567    0
25079    1
18707    1
15200    0
5857     1
Name: sentiment, Length: 10000, dtype: int64


**Building Long Short Term Memory Model**

In [ ]:
# Building the model

In [ ]:
model= Sequential() #to stack layers linearly
model.add(Embedding(input_dim=5000,output_dim=128,input_length=200,input_shape=(200,))) # to convert data in its vector form, embedding should be the first layer of model
# input dimention represents 5000 most frequent selected tokens
# output dimention represents the vector representation or dimention of verctor,means each vector should be of 128 dimentions
# input length 200 reresents the max lenth of each datapoint,means each review should have equall 200 tokens
# input shape represents shape of layer
model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2))
# 128 represents 128 units,means each datapoint/review will be represented as 1 vector of 128 dimensions.
# dropout will randomly shutdown few neurons so that model donot overfit and generaize,in that way model donot reply on specific neurons/features
#drop out is applied to input/output layer whil recurrent drop out is applies to hidden layers
# droput= 0.2  represents that 20 percent of training data will be switch off or flow as 0..only applies during training.
#Flow:
#1) 5000 most frequent words will be converted into tokens and integers,integers just act like ids,embedding has the contexual meaning
#2) all these tokens will be converted into vectors, each vector has dimension of 128
#3) suppose we have 200 tokens so the shape will be then (200,128) means 200 tokens ,each token of 128 dimension
#4)then lstm(128) units means each data point will be then converted into 1 vector of 128 dimension
model.add(Dense(1, activation='sigmoid')) # dense shows fully connected neural network, 1 represents 1 out put neuron in output layer.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 200, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# none reresents batch size which is flexible and can be replaced by a any number
# batch size represents no. of samples being processed at once
# 200, 128 means each input has 200 tokens and each token is a 128 dimensional vector
# param represents parameters or trainable values

In [ ]:
# Compiling the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
# optimizer changes weights and biases at every epochs for imporving prediction
# binary cross entropy loss is suitable for binary classification
# accuracy metrics to check model accuracy

**Model Training**

In [ ]:
model.fit(X_train,Y_train,epochs=5,batch_size=64, validation_split=0.2)
# batch size represents no. of training samples being processed at once
# validation split =0.2 will reserve and use 20 percent of data for model assesement during training after every epochs to detect and prevent model from overfitting
# epochs are iterations, 1 epochs is 1 complet forward and backward pass

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 348s 686ms/step - accuracy: 0.7096 - loss: 0.5438 - val_accuracy: 0.8253 - val_loss: 0.4108
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 334s 666ms/step - accuracy: 0.8450 - loss: 0.3733 - val_accuracy: 0.8590 - val_loss: 0.3335
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 380s 664ms/step - accuracy: 0.8734 - loss: 0.3096 - val_accuracy: 0.8587 - val_loss: 0.3427
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 377s 654ms/step - accuracy: 0.8896 - loss: 0.2749 - val_accuracy: 0.8746 - val_loss: 0.3153
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 387s 664ms/step - accuracy: 0.8979 - loss: 0.2556 - val_accuracy: 0.8754 - val_loss: 0.3297


**Model Evaluation**

In [ ]:
loss,accuracy= model.evaluate(X_test,Y_test)
print(f"The test loss is: {loss}")
print(f"Thes test accuracy is: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 30s 92ms/step - accuracy: 0.8771 - loss: 0.3136
The test loss is: 0.31040313839912415
Thes test accuracy is: 0.8802000284194946


**Building a Predictive System**

In [ ]:
def predict_sentiment(review): # creating funtion name as predict_sequence that takes review as input
  sequence= tokenizer.texts_to_sequences(review) #converting text to tokens ang generating integers of input reviews
  paded_sequence= pad_sequences(sequence,maxlen=200) #will ensure every sequence/every review has equall 200 tokens
  prediction= model.predict(paded_sequence)
  sentiment= "Positive Review" if prediction[0][0]>0.5 else "Negative Review"  #we will have probailities [[0.5],[0.4]],not exact prediction like in ml
  # we will compare first and 2nd value with threashhold 0.5, if first value is greater then 0.5 means the review is positive,if 2nd value is greater means the review is negative
  return sentiment

In [ ]:
#Example usage,calling the function
new_review= "this movie is great.I love the movie.Fantastic"
sentiment= predict_sentiment([new_review]) # the square bracket is necessary,not using them will let model take every word separatly as a separate input ,not the part of single review
# deep learning models are designed to do task in batches on large data set not one by one so it always expects list of reviews,multiple reviws or batches
# we will use [] to show it as a list even if we have only one data point or 1 instance of review
print(f" The Review is : {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 512ms/step
 The Review is : Positive Review


In [ ]:
#Example usage,calling the function
new_review= "this movie is bad.I hate the actors.not worth it"
sentiment= predict_sentiment([new_review])
print(f" The Review is : {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
 The Review is : Negative Review


In [ ]:
#Example usage,calling the function
new_review= "this movie is good"
sentiment= predict_sentiment([new_review])
print(f" The Review is : {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
 The Review is : Positive Review


In [ ]:
#Example usage,calling the function
new_review= "lovely"
sentiment= predict_sentiment([new_review])
print(f" The Review is : {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
 The Review is : Positive Review


In [ ]:
#Example usage,calling the function
new_review= "Better to stop making movies and wasting money"
sentiment= predict_sentiment([new_review])
print(f" The Review is : {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
 The Review is : Negative Review
